# BT4221 Group 20: Optimising Airfare Booking with Predictive Analytics

Group Members:

Gong Yongjia A0286144X

Jody Tng Jin Zi A0238195W

Wu Shuhan A0266501L

Zhou Jingchu Jeslyn A0275993H




# Data Loading & Exploration

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark Sesison
spark = SparkSession.builder.appName("FlightPrices").getOrCreate()

# Load Dataset
flight_df = spark.read.csv("/content/drive/MyDrive/BT4221/flight.csv", header=True, inferSchema=True)

# Show a sample (5 rows) of the data
flight_df.show(5, truncate=False)

# Display schema
flight_df.printSchema()

# Total Count
flight_df.count()

+--------------------------------+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------+--------------+-------------------+---------------------------------+------------------------------------------------------------+-------------------------------+------------------------------------------------------------+--------------------------+----------------------------+-------------------+-------------------+------------------------------+-------------------------+----------------+-----------------+
|legId                           |searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|seatsRemaining|totalTravelDistance|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw                                    |segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw                          

2000000

# Data Cleaning & Pre-processing

In [ ]:
from pyspark.sql.functions import col, sum, isnan
from pyspark.sql.types import NumericType

# Build expressions to count missing values (null and NaN for numeric columns; null only otherwise)
missing_exprs = []
for field in flight_df.schema.fields:
    if isinstance(field.dataType, NumericType):
        # For numeric columns, count both nulls and NaNs
        expr = sum((col(field.name).isNull() | isnan(col(field.name))).cast("int")).alias(field.name)
    else:
        # For non-numeric columns, count only nulls
        expr = sum(col(field.name).isNull().cast("int")).alias(field.name)
    missing_exprs.append(expr)

# Compute missing values count per column
missing_counts = flight_df.select(*missing_exprs)
missing_counts.show()

+-----+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------+--------------+-------------------+---------------------------------+------------------------+-------------------------------+----------------------+--------------------------+----------------------------+-------------------+-------------------+----------------------------+-------------------------+----------------+-----------------+
|legId|searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|seatsRemaining|totalTravelDistance|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw|segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw|segmentsArrivalAirportCode|segmentsDepartureAirportCode|segmentsAirlineName|segmentsAirlineCode|segmentsEquipmentDescription|segmentsDurationInSeconds|segmentsDistance|segmentsCabinCode|
+---

In [ ]:
# Remove rows with null values

# clean_flight_data = flight_df.dropna(subset = )


# Drop duplicates
flight_df = flight_df.dropDuplicates()

# Final Count
flight_df.count()




2000000